In [3]:
%reset -f

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import ee
ee.Authenticate()


Mounted at /content/drive


In [21]:
import os
import ee

ee.Initialize(project='ee-andrewfullhart')

ic = ee.ImageCollection('NASA/NEX-GDDP')

inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/GHCN_Historical_Annual_Mean_MEANP_Trends_Results.csv'

study_area = ee.FeatureCollection('users/andrewfullhart/SW_Study_Area')

ndays_months = ee.List([31., 28.25, 31., 30., 31., 30., 31., 31., 30., 31., 30., 31.])
order_months = ee.List([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])


#"""Append nothing to output file name if running this block."""

#with open(inFILE) as f:
#  lines = f.readlines()

#points_ft_list = ee.List([])
#points_label_list = ee.List([])
#for line in lines[1:]:
#  row = line.strip('\n').split(',')
#  lon = float(row[1])
#  lat = float(row[2])
#  ft = ee.Feature(None, {}).setGeometry(ee.Geometry.Point(lon, lat))
#  points_ft_list = points_ft_list.add(ft)
#  points_label_list = points_label_list.add(row[0])

#points_fc = ee.FeatureCollection(points_ft_list)


"""Append _CLINET to output file name if running this block."""

points_ft_list = ee.List([])
points_label_list = ee.List([])
parFOLDER = '/content/drive/My Drive/Colab Notebooks/CLIGEN/2015parfiles/2015parfiles/2015parfiles'
par_files = os.listdir(parFOLDER)
for parf in par_files:
  if parf[:2] in ['az', 'nv', 'nm', 'ut', 'US']:
    stationID = parf.strip('.par')
    if stationID[0] == 'z':
       stationID = 'a' + stationID
    with open(os.path.join(parFOLDER, parf)) as f:
      lines = f.readlines()
    for line in lines:
      label = line[:8]
      label = label.rstrip().lstrip()
      dataline = line[8:]
      if label == 'LATT=':
        lat = float(line.split('=')[1].strip('LONG').strip())
        lon = float(line.split('=')[2].strip('YEARS').strip())
    ft = ee.Feature(None, {}).setGeometry(ee.Geometry.Point(lon, lat))
    points_ft_list = points_ft_list.add(ft)
    points_label_list = points_label_list.add(stationID)

points_fc = ee.FeatureCollection(points_ft_list)



start_year_global = 1974
end_year_global = 2013
years_list_global = ee.List.sequence(start_year_global, end_year_global)
years_index_global = ee.List.sequence(0, years_list_global.size().subtract(1))

def pointlabel_fn(point):
  point = ee.Feature(point).geometry()
  a_strng = ee.Number(point.coordinates().get(0)).format('%.3f')
  b_strng = ee.String('_')
  c_strng = ee.Number(point.coordinates().get(1)).format('%.3f')
  point_strng = a_strng.cat(b_strng).cat(c_strng)
  point_strng = point_strng.replace('\\.', '_', 'g').slice(1)
  return point_strng

point_labels_global = points_fc.toList(points_fc.size()).map(pointlabel_fn)

def year_fn(year):
  year = ee.Number(year)
  start = ee.Date.fromYMD(ee.Number(year), 1, 1)
  end = ee.Date.fromYMD(ee.Number(year).add(1), 1, 1)
  year_ic = ic.filterDate(start, end).select('pr')
  modelfilter = ee.Filter.Or(
                ee.Filter.eq('scenario', 'historical'),
                ee.Filter.eq('scenario', 'rcp45'))
  year_ic = year_ic.filter(modelfilter)
  year_ic = year_ic.filter(ee.Filter.eq('model', 'CCSM4'))

  def point_fn(point):
    point = point.geometry()
    a_strng = ee.Number(point.coordinates().get(0)).format('%.3f')
    b_strng = ee.String('_')
    c_strng = ee.Number(point.coordinates().get(1)).format('%.3f')
    point_strng = a_strng.cat(b_strng).cat(c_strng)
    point_strng = point_strng.replace('\\.', '_', 'g').slice(1)
    data_list = year_ic.getRegion(point, 500).slice(1)

    def feature_fn(l):
      l = ee.List(l)
      date_str = ee.String(ee.String(l.get(0)).split('_').get(-1))
      year = ee.Number(date_str.slice(0, 4))
      month = ee.Number(date_str.slice(4, 6))
      prcp = ee.Number(l.get(4))
      prop_dict = {'year':year, 'month':month, 'precip':prcp}
      point_ft = ee.Feature(None, prop_dict)
      return point_ft

    data_fc = ee.FeatureCollection(data_list.map(feature_fn))
    nonzero_fc = data_fc.filter(ee.Filter.gt('precip', 0))
    number = ee.Number(nonzero_fc.reduceColumns(ee.Reducer.mean(), ['precip']).get('mean')).multiply(86400)
    number_ft = ee.Feature(None, {'point':point_strng,'mean':number})
    return number_ft

  means_fc = ee.FeatureCollection(points_fc.map(point_fn))
  return means_fc

means_fc_list = ee.FeatureCollection(years_list_global.map(year_fn)).toList(years_list_global.size())

def flatten_fn(mean_fc):
  mean_fc = ee.FeatureCollection(mean_fc)
  data_list = ee.List(mean_fc.reduceColumns(ee.Reducer.toList(), ['mean']).get('list'))
  #prop_dict = ee.Dictionary(point_labels_global.zip(data_list).flatten())
  prop_dict = ee.Dictionary(points_label_list.zip(data_list).flatten())
  return  ee.Feature(None, prop_dict)

out_fc = ee.FeatureCollection(means_fc_list.map(flatten_fn))

task = ee.batch.Export.table.toDrive(collection=out_fc,
                           description='NEX_Historical_Annual_Mean_MEANP_Trends',
                           folder='GEE_Downloads')

task.start()




In [5]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

geeFILE = '/content/drive/My Drive/GEE_Downloads/NEX_Historical_Annual_Mean_MEANP_Trends.csv'
outFILE = '/content/drive/My Drive/Colab Notebooks/NEX_Historical_Annual_Mean_MEANP_Trends_Results_CLINET.csv'
dataFILE = '/content/drive/My Drive/Colab Notebooks/NEX_Historical_Annual_Mean_MEANP_Trends_Data_CLINET.csv'

"""For GHCN blocks"""
#inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/GHCN_Historical_Annual_Mean_MEANP_Trends_Results.csv'
"""For _CLINET blocks"""
inFILE = '/content/drive/My Drive/Colab Notebooks/Script Input Files/US_CLIGEN_Coords.csv'

with open(inFILE) as f:
  lines = f.readlines()

stationIDjoin_Dict = {}
for line in lines[1:]:
  row = line.strip('\n').split(',')
  lon = row[1]
  lat = row[2]
  stationID = row[0]
  stationIDjoin_Dict[stationID] = [lon, lat]

df = pd.read_csv(geeFILE)
data_df = pd.DataFrame()
point_list = df.columns.values[1:-1]

save_data = []
for i, point in enumerate(point_list):
  print(point)
  y = df[point]
  x = np.arange(0, 40)
  X = sm.add_constant(x)
  model = sm.OLS(y, X)
  results = model.fit()

  slope = results.params[1]
  #parts = point.split('_')
  #lon = '-' + parts[0] + '.' + parts[1]
  #lat = parts[2] + '.' + parts[3]
  lon = stationIDjoin_Dict[point][0]
  lat = stationIDjoin_Dict[point][1]
  save_data.append([point, lon, lat, str(slope)])

  data_df[point] = y

data_df.to_csv(dataFILE)

with open(outFILE, 'w') as fo:
  fo.write('stationID,x,y,slope\n')
  for elem in save_data:
    fo.write(','.join([e for e in elem]) + '\n')



az020080
az020159
az020287
az020632
az020750
az020808
az020949
az021001
az021314
az021330
az021353
az021654
az022109
az022140
az022664
az022754
az023010
az023160
az023303
az023393
az023683
az023961
az024453
az024675
az024702
az024829
az025412
az025635
az025924
az026132
az026250
az026323
az026468
az026481
az026796
az026801
az027281
az027390
az027435
az027716
az028012
az028162
az028214
az028396
az028499
az028619
az028820
az028865
az029158
az029287
az029334
az029359
az029439
az029542
az029645
az029654
nm290234
nm290245
nm290377
nm290417
nm290600
nm290640
nm290692
nm290992
nm291063
nm291138
nm291286
nm291469
nm291475
nm291480
nm291630
nm291647
nm291664
nm291813
nm291887
nm291910
nm291963
nm292030
nm292510
nm292608
nm292700
nm292785
nm292820
nm292837
nm292848
nm292854
nm292865
nm293060
nm293157
nm293488
nm293577
nm293649
nm293706
nm294009
nm294026
nm294346
nm294369
nm294426
nm294742
nm294856
nm295084
nm295273
nm295560
nm295617
nm295937
nm295960
nm295965
nm296619
nm296676
nm296687
nm297008
n

<ipython-input-5-3f243dbcf8f3>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[point] = y
<ipython-input-5-3f243dbcf8f3>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[point] = y
<ipython-input-5-3f243dbcf8f3>:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_df[point] = y
<